In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [9]:
df = pd.read_csv("heart_indicators_clean_data.csv", sep = ",", error_bad_lines=False) 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 23387: expected 22 fields, saw 39\n'
b'Skipping line 58493: expected 22 fields, saw 23\n'
b'Skipping line 70197: expected 22 fields, saw 28\nSkipping line 93581: expected 22 fields, saw 31\n'
b'Skipping line 105276: expected 22 fields, saw 25\nSkipping line 128678: expected 22 fields, saw 34\n'
b'Skipping line 163782: expected 22 fields, saw 34\n'
b'Skipping line 175477: expected 22 fields, saw 26\n'
b'Skipping line 222272: expected 22 fields, saw 37\n'
b'Skipping line 245664: expected 22 fields, saw 27\n'
b'Skipping line 292469: expected 22 fields, saw 24\n'
b'Skipping line 315869: expected 22 fields, saw 34\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,8) have mixed types.Spec

In [11]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('HeartDiseaseorAttack'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('HeartDiseaseorAttack'))
test_labels = np.array(test_df.pop('HeartDiseaseorAttack'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [17]:
train_features

array([[0.0, 0.0, 1.0, ..., 7.0, 6.0, 3.0],
       [0.0, 0.0, 1.0, ..., 8.0, 5.0, 3.0],
       [1.0, 1.0, 1.0, ..., 7.0, 6.0, 7.0],
       ...,
       [1.0, 1.0, 1.0, ..., 10.0, 6.0, 6.0],
       [1.0, 1.0, 1.0, ..., 8.0, 5.0, 2.0],
       [0.0, 0.0, 1.0, ..., 10.0, 6.0, 6.0]], dtype=object)

In [12]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(number_of_hidden_layers=1, activation_function='relu', final_activation_function='sigmoid', learning_rate=1e-3):
  layers = [keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],))]
  for layer in range(number_of_hidden_layers):
    layers.append(keras.layers.Dense(16, input_dim=16, activation=activation_function))

  layers.append(keras.layers.Dropout(0.5))
  layers.append(keras.layers.Dense(1, activation=final_activation_function) )

  model = keras.Sequential(layers)

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=METRICS)

  return model


In [17]:
#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
# clf.fit(X_train,y_train)

# y_pred = clf.predict(X_test)

clf.fit(train_features,train_labels)

y_pred = clf.predict(val_features)

In [19]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

Repeat

In [22]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

BUFFER_SIZE = 100000

def make_ds(features, labels):
  # ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  
  ds = tf.data.Dataset.from_tensor_slices((features, labels)).cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

ValueError: ignored

In [ ]:
val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

In [ ]:
model = make_model(number_of_hidden_layers=1)

resampled_history = model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks=[early_stopping],
    validation_data=val_ds)